#Step 1 -> **Download data**

In [1]:
# Download zip file
import os
import requests
import zipfile
from pathlib import Path

# Set Data path
data_path = Path("/data")
image_path = data_path / "food"

# if image folder doesn't exists download from github
if image_path.is_dir():
  print(f"{image_path} already exists.")
else:
  print(f"create new directory {image_path}")
  image_path.mkdir(parents=True, exist_ok=True)

# open temp zip file and download data into tmp zip file and extract it
with open(data_path / "food.zip", "wb") as f:
  req = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading data")
  f.write(req.content)

# open zip file and extract all content
with zipfile.ZipFile(data_path / "food.zip", "r") as f:
  print(f"Unzipping zip file")
  f.extractall(image_path)

#remove zip file
os.remove(data_path / "food.zip")

create new directory /data/food
Unzipping zip file


In [2]:
import os

os.makedirs("dnnmodule", exist_ok=True)

# Step 2 -> Create datasets and dataloaders

In [3]:
#%%writefile dnnmodule/data_setup.py

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  #use ImageFolder to create datasets
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  #classnames
  class_names = train_data.classes

  #convert images into dataloader batch
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True
  )

  return train_dataloader, test_dataloader, class_names


# Step 3 -> Create Model

In [15]:
#%%writefile dnnmodule/model.py

import torch
from torch import nn

"""
We are building TinyCnn model with 2 cnn block and 1 classifier block
"""
class TinyCNN(nn.Module):
  def __init__(self, input_shape:int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0
                  ),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0
                  ),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x


# Step 4 -> create engine -> train_step, test_step , train

In [16]:
#%%writefile dnnmodule/engine.py

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  model.train()

  # assign train loss and accuracy value
  train_loss, train_acc = 0, 0

  # loop through data loaders
  for batch, (X,y) in enumerate(dataloader):
    # Set target to device
    X,y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. calculate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. optimize zero grad
    optimizer.zero_grad()

    # 4. loss backward
    loss.backward()

    # 5. optimizer step
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  #adjust metrics 
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)

  return train_loss, train_acc


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  model.eval()

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch, (X,y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      # 1. forward pass
      y_pred = model(X)

      # 2. calculate loss
      loss = loss_fn(y_pred, y)
      test_loss += loss.item()

      # 3. calculate accurancy
      test_pred_labels = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
  
  # Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device)-> Dict[str, List]:
  results = {"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[]}

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=train_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)
    print(
        f"Epoch: {epoch+1} | "
        f"train_loss: {train_loss:.4f} | "
        f"train_acc: {train_acc:.4f} | "
        f"test_loss: {test_loss:.4f} | "
        f"test_acc: {test_acc:.4f}"
    )

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results


# Step 5 -> save model

In [19]:
#%%writefile going_modular/utils.py

import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir:str,
               model_name: str):
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "Model name should endwith .pth .pt"
  model_save_path = target_dir_path / model_name

  print(f"saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)


# Final step run trainning

In [22]:
import os
import torch

from torchvision import transforms

NUM_EPOCHS = 20
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001


train_dir = "/data/food/train"
test_dir = "/data/food/test"

device = "cuda" if torch.cuda.is_available() else "cpu"

data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

model = TinyCNN(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

train(model=model,
      train_dataloader=train_dataloader,
      test_dataloader=test_dataloader,
      loss_fn=loss_fn,
      optimizer=optimizer,
      epochs=NUM_EPOCHS,
      device=device)

save_model(model=model,
           target_dir="models",
           model_name="tinyvgg.pth")


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0988 | train_acc: 0.4492 | test_loss: 1.0906 | test_acc: 0.4258
Epoch: 2 | train_loss: 1.1156 | train_acc: 0.3047 | test_loss: 1.1077 | test_acc: 0.3047
Epoch: 3 | train_loss: 1.1043 | train_acc: 0.3047 | test_loss: 1.0900 | test_acc: 0.5430
Epoch: 4 | train_loss: 1.0948 | train_acc: 0.3359 | test_loss: 1.0879 | test_acc: 0.6016
Epoch: 5 | train_loss: 1.0836 | train_acc: 0.4453 | test_loss: 1.0559 | test_acc: 0.4805
Epoch: 6 | train_loss: 1.0785 | train_acc: 0.3828 | test_loss: 1.0290 | test_acc: 0.5312
Epoch: 7 | train_loss: 1.0591 | train_acc: 0.3906 | test_loss: 1.0715 | test_acc: 0.3477
Epoch: 8 | train_loss: 1.0601 | train_acc: 0.3398 | test_loss: 1.0243 | test_acc: 0.4453
Epoch: 9 | train_loss: 0.9152 | train_acc: 0.5742 | test_loss: 0.9815 | test_acc: 0.4414
Epoch: 10 | train_loss: 0.9721 | train_acc: 0.4531 | test_loss: 0.8766 | test_acc: 0.6484
Epoch: 11 | train_loss: 0.8638 | train_acc: 0.5977 | test_loss: 0.8301 | test_acc: 0.6055
Epoch: 12 | train_l